# 음성인식 오류율 측정 예제

## 환경셋업

In [1]:
# Run this cell to automatically clone the repo
import sys
import os

if 'google.colab' in sys.modules:
    if not os.path.exists("/content/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" /content/a1003
    %cd /content/a1003/local
else:
    if not os.path.exists("~/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" ~/a1003
    %cd ~/a1003/local

/content/a1003/local


## 개별 문장에 대한 오류율 측정




다음 코드는 두 개의 한국어 문장(정답, 음성인식결과)에 대하여
edit distance(최소편집거리)를 계산하여 WER과 CER을 측정하는 코드입니다.

Edit distance를 계산하기 위해서 `editdistance` 모듈을 import하고,
두 개의 문장을 각 변수에 저장합니다.


In [2]:
import editdistance

ref_text = "오늘 서울의 날씨가 어때"
hyp_text = "음 오늘의 날씨 가 어때"

WER을 측정하기 위해서 문장을 단어단위로 분리하여 리스트에 저장하빈다.

In [3]:
ref = ref_text.split()
hyp = hyp_text.split()

print(ref, hyp, sep='\n')

['오늘', '서울의', '날씨가', '어때']
['음', '오늘의', '날씨', '가', '어때']


Edit distance를 계산합니다. `editdistance.eval()` 함수는 두개의 리스트를 입력받아서,
두 리스트 간의 editdistance를 반환하는 함수입니다.

In [4]:
E = editdistance.eval(ref, hyp)
N = len(ref)
WER = E/N*100
print(f"N={N}, E={E}, WER={WER}")

N=4, E=4, WER=100.0


CER을 측정하기 위해서 문장을 각 글자의 리스트로 변환합니다.

In [5]:
ref = list(ref_text)
hyp = list(hyp_text)
print(ref, hyp, sep='\n')

['오', '늘', ' ', '서', '울', '의', ' ', '날', '씨', '가', ' ', '어', '때']
['음', ' ', '오', '늘', '의', ' ', '날', '씨', ' ', '가', ' ', '어', '때']



글자 단위로 분리된 리스트(`ref`, `hyp`)에 대해서 같은 방법으로 editdistance를 측정합니다.
공백문자도 하나의 글자로 처리된 것도 확인하세요.


In [6]:
E = editdistance.eval(ref, hyp)
N = len(ref)
CER = E/N*100
print(f"N={N}, E={E}, CER={CER}")

N=13, E=5, CER=38.46153846153847


이번에는 공백문자를 무시하고(즉, 띄어쓰기 오류를 완전히 무시하고) CER을 측정해 봅니다.
앞에서 글자단위로 분리한 것과 다르게 이번에는 공백문자를 미리 제거합니다.

In [7]:
ref = list(''.join(ref_text.split()))
hyp = list(''.join(hyp_text.split()))
print(ref, hyp, sep='\n')

['오', '늘', '서', '울', '의', '날', '씨', '가', '어', '때']
['음', '오', '늘', '의', '날', '씨', '가', '어', '때']


In [8]:
E = editdistance.eval(ref, hyp)
N = len(ref)
CER = E/N*100
print(f"N={N}, E={E}, CER={CER}")

N=10, E=3, CER=30.0


## 여러 문장에 대한 평균 오류율 측정

지금까지는 한개의 {정답,인식결과} 쌍에 대해서 WER, CER을 측정해보았습니다.
실제로는 매우 많은 {정답,인식결과} 쌍에 대해서 오류율을 계산하고 전체의 오류율을 성능지표로 사용합니다.
이를 위해서 `kaldi`와 `ESPnet`은 다음과 같은 형식으로 여러 파일에 대한 정답전사문과 인식결과를 파일로 저장해둡니다.

In [9]:
!cat data/ref.txt

KsponSpeech_E00001 어 일단은 억지로 과장해서 이렇게 하는 것보다 진실된 마음으로 이걸 어떻게 전달할 수 있을까 공감을 시킬 수 있을까 해서 좀
KsponSpeech_E00002 혼인 신고를 또 해야 되잖아 
KsponSpeech_E00003 약간 젊은 엄마 같은 느낌이야 
KsponSpeech_E00004 응 근데 오늘 일단 밥 먹고 이것 저것 하다가 시간되면 뭐 가는 거고 안 되면
KsponSpeech_E00005 아 우린 또 그런 거 안 하잖아 어 그치 
KsponSpeech_E00006 아 근데 진짜 감튀 먹고 있는데 데려온 거 심했다
KsponSpeech_E00007 그래서 저승에서 그 애기를 찾아서 막 돌아다니는 내용인데 
KsponSpeech_E00008 그런가 
KsponSpeech_E00009 그 너가 매운 거를 잘 못먹어서 못 먹기도 하고
KsponSpeech_E00010 그래서 나는 지금 7월달에는 사람들이 돈 30만 원씩 뭐 달마다 30만 원씩 모아서 지금 그 태국여행 가기로 했거든 


In [10]:
!cat data/hyp.txt

KsponSpeech_E00001 어 일단은 억지로 과장에서 이렇게 하는 것보다 진실된 마음으로 이걸 어떻게 전달을 할 수 있을까 공감을 시킬 수 있을까 해서 좀
KsponSpeech_E00002 혼인 신고를 또 해야 되잖아
KsponSpeech_E00003 약간 젊은 엄마 같은 느낌이야
KsponSpeech_E00004 음 근데 오늘 일단 밥 먹고 이것저것 다 따라 하다가 시간 되면 >뭐 가는 거고 안 되면
KsponSpeech_E00005 아무랜도 그런 거 알잖아 어 그지
KsponSpeech_E00006 아 근데 진짜 감기 먹고 있는데 데려온 구심했다
KsponSpeech_E00007 그래서 저순에서 그 애기를 찾아서 막 돌아다니는 내용인데
KsponSpeech_E00008 그런가
KsponSpeech_E00009 근데 너가 매운 거를 잘 못 먹었 못 먹기도 하고
KsponSpeech_E00010 그래도 나는 지금 칠 월 달에는 사람들이 조 삼십만 원씩 뭐 달만 삼십 하십 모아서 지금 그 태홍된가 그랬거든


이 두개의 파일을 읽어서 utternace-id 가 같은 문장간의 editdistance를 출력하는 파이선 코드가 `uttwer.py`, `uttcer.py` 파일입니다.
다음과 같은 방법으로 실행합니다.

In [11]:
!python uttwer.py data/ref.txt data/hyp.txt

KsponSpeech_E00001 15.00 3 20
KsponSpeech_E00002  0.00 0 5
KsponSpeech_E00003  0.00 0 5
KsponSpeech_E00004 46.67 7 15
KsponSpeech_E00005 66.67 6 9
KsponSpeech_E00006 33.33 3 9
KsponSpeech_E00007 12.50 1 8
KsponSpeech_E00008  0.00 0 1
KsponSpeech_E00009 33.33 3 9
KsponSpeech_E00010 66.67 12 18
N= 99 E= 35 WER= 35.35


In [12]:
!python uttcer.py data/ref.txt data/hyp.txt

KsponSpeech_E00001  4.00 2 50
KsponSpeech_E00002  0.00 0 11
KsponSpeech_E00003  0.00 0 12
KsponSpeech_E00004 17.24 5 29
KsponSpeech_E00005 42.86 6 14
KsponSpeech_E00006 10.53 2 19
KsponSpeech_E00007  4.17 1 24
KsponSpeech_E00008  0.00 0 3
KsponSpeech_E00009 22.22 4 18
KsponSpeech_E00010 38.30 18 47
N= 227 E= 38 CER= 16.74
